In [1]:
import javalang

java_code = r"""
public class Test{
    protected void saveAsSVG(File f) throws IOException {
        File file = f;
        if (file == null) {
            JFileChooser fileChooser = new JFileChooser();
            fileChooser.setCurrentDirectory(this.defaultDirectoryForSaveAs);
            FileNameExtensionFilter filter = new FileNameExtensionFilter(
                    localizationResources.getString("SVG_Files"), "svg");
            fileChooser.addChoosableFileFilter(filter);
            fileChooser.setFileFilter(filter);

            int option = fileChooser.showSaveDialog(this);
            if (option == JFileChooser.APPROVE_OPTION) {
                String filename = fileChooser.getSelectedFile().getPath();
                if (isEnforceFileExtensions()) {
                    if (!filename.endsWith(".svg")) {
                        filename = filename + ".svg";
                    }
                }
                file = new File(filename);
                if (file.exists()) {
                    String fileExists = localizationResources.getString(
                            "FILE_EXISTS_CONFIRM_OVERWRITE");
                    int response = JOptionPane.showConfirmDialog(this,
                            fileExists,
                            localizationResources.getString("Save_as_SVG"),
                            JOptionPane.OK_CANCEL_OPTION);
                    if (response == JOptionPane.CANCEL_OPTION) {
                        file = null;
                    }
                }

            }
        }

        if (file != null) {
            // use reflection to get the SVG string
            String svg = generateSVG(getWidth(), getHeight());
            BufferedWriter writer = null;
            Exception originalException = null;
            try {
                writer = new BufferedWriter(new FileWriter(file));
                writer.write("<!DOCTYPE svg PUBLIC \"-//W3C//DTD SVG 1.1//EN\" \"http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd\">\n");
                writer.write(svg + "\n");
                writer.flush();
            } catch (Exception e) {
                originalException = e;
            }
            try {
                if (writer != null) {
                    writer.close();
                }
            } catch (IOException ex) {
                RuntimeException th = new RuntimeException(ex);
                if (originalException != null)
                    th.addSuppressed(originalException);
                throw th;
            }
        }
    }
}

"""
lines = java_code.splitlines()
tree = javalang.parse.parse(java_code)
clazz = tree.types[0]
method = clazz.methods[0]
print(method.position[0], method.name)
# 遍历method，找到所有语句的start
for node in method.body:
    start = node.position[0]
    print(start, lines[start - 1].strip())

3 saveAsSVG
4 File file = f;
5 if (file == null) {
37 if (file != null) {


In [5]:
# iter method
def get_method_end(method):
    def f(node):
        if hasattr(node, 'children'):
            for child in node.children:
                yield from f(child)

        if isinstance(node, list):
            for child in node:
                yield from f(child)

        if hasattr(node, 'position'):
            if node.position is not None:
                try:
                    yield node.position[0], lines[node.position[0] - 1]
                except Exception as e:
                    print(node.position)
    stmt_set = set(f(method))
    return max(stmt_set, key=lambda x: x[0]) + 1

{(3, '    protected void saveAsSVG(File f) throws IOException {'),
 (4, '        File file = f;'),
 (5, '        if (file == null) {'),
 (6, '            JFileChooser fileChooser = new JFileChooser();'),
 (7,
  '            fileChooser.setCurrentDirectory(this.defaultDirectoryForSaveAs);'),
 (8,
  '            FileNameExtensionFilter filter = new FileNameExtensionFilter('),
 (9,
  '                    localizationResources.getString("SVG_Files"), "svg");'),
 (10, '            fileChooser.addChoosableFileFilter(filter);'),
 (11, '            fileChooser.setFileFilter(filter);'),
 (13, '            int option = fileChooser.showSaveDialog(this);'),
 (14, '            if (option == JFileChooser.APPROVE_OPTION) {'),
 (15,
  '                String filename = fileChooser.getSelectedFile().getPath();'),
 (16, '                if (isEnforceFileExtensions()) {'),
 (17, '                    if (!filename.endsWith(".svg")) {'),
 (18, '                        filename = filename + ".svg";'),
 (21,

In [15]:
import os
def get_class_imports(source_folder, class_names: list):
    # 递归遍历source文件夹，找到所有的class文件
    class_map = {} # {class_name: [package_reference, ...]}
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith(".java"):
                class_name = file[:-5]
                root = root.replace("\\", ".")
                root = root.replace("/", ".")
                if "src.test.java" in root:
                    continue
                ref = root.split("src.main.java.")[1]
                if class_name not in class_map:
                    class_map[class_name] = [ref]
                else:
                    class_map[class_name].append(ref)
    imports = []
    for name in class_names:
        if name not in class_map:
            return None

        refs = class_map[name]
        for ref in refs:
            imports.append(f"import {ref}.{name};")
    return imports
get_class_imports(r"D:\IDEA_Projects\TestJavaCode\src", ["Plot"])

['import org.jfree.chart.plot.Plot;']